# 实验八 矩阵特征值及特征向量计算

- 正幂法

In [220]:
import numpy as np

def power_iteration(A, m0=1, u0=None, eps=1e-8, max_steps=500, verbose=False):
    """正幂法（power iteration a.k.a. the power method）计算矩阵 A 的按模最大特征值、特征向量
    
    Args:
        A:   np_array_like 待求特征值的矩阵 (nxn)
        m0:  float 初始特征值
                default m0=1
        u0: np_array_like 初始特征向量（n）
                default u0=None: 取 u0 = (1, 1, ..., 1)
        eps: float 精度要求
                default eps=1e-8
        max_steps: int 最大迭代次数
                default max_steps=1000
        verbose: bool, 若为 True 则打印出每一步的结果
                default verbose=False
        
    Returns:
        (m, u, k): 在 max_steps 次迭代以内得到第一组的满足 eps 的结果
            m: float 所求主特征值
            u: np.array 相应的特征向量
            k: int 迭代次数
        
    Raises:
        ValueError: 参数 A 不是方阵，
                    或 A 和给定 u0 尺寸不匹配
        Exception:  无法在max_steps 次迭代以内得到满足精度 eps 的结果
    """
        
    _shape = np.shape(A)
    if len(_shape) < 2 or _shape[0] != _shape[1]:
        raise ValueError(f"unexpected A, shape: {_shape}")
    
    if u0 is not None:
        if len(u0) != _shape[0]:
            raise ValueError(f"A (shape={_shape}) and u0 (len={len(u0)}) not match")
    else:  # not u0
        u0 = np.ones(_shape[0])
        
    m = m0
    u = u0

    for k in range(int(max_steps)):
        if verbose:
            print(k, m, u)
        
        m_prev = m
        
        v = np.dot(A, u)
        mi = np.argmax(np.abs(v))
        m = v[mi]
        u = v / m
        
        if abs(m - m_prev) <= eps:
            break
            
    else:
        raise Exception(f"cannot reach eps ({eps}) after max_steps ({max_steps}). "
                        f"The last result: m = {m}, u={u}")
        
    if verbose:
        print('result of power_iteration:', m, u, k+1)

    return m, u, k+1

算一个课本上的例子（P159 例1）：

In [221]:
A = [[2, 3, 2], [10, 3, 4], [3, 6, 1]]
power_iteration(A, eps=1e-8, verbose=True)

0 1 [1. 1. 1.]
1 17.0 [0.41176471 1.         0.58823529]
2 9.470588235294118 [0.52795031 1.         0.82608696]
3 11.58385093167702 [0.49276139 1.         0.72600536]
4 10.831635388739945 [0.50200485 1.         0.75743775]
5 11.049799514875502 [0.49945569 1.         0.74783731]
6 10.985906091381928 [0.50014864 1.         0.75061668]
7 11.003953118800313 [0.49995948 1.         0.74982713]
8 10.998903290037243 [0.50001105 1.         0.75004801]
9 11.000302582696586 [0.49999699 1.         0.74998675]
10 10.999916852432536 [0.50000082 1.         0.75000364]
11 11.000022790833176 [0.49999978 1.         0.749999  ]
12 10.999993763594336 [0.50000006 1.         0.75000027]
13 11.000001704609195 [0.49999998 1.         0.74999993]
14 10.999999534421143 [0.5        1.         0.75000002]
15 11.000000127100696 [0.5        1.         0.74999999]
16 10.999999965313513 [0.5  1.   0.75]
17 11.00000000946407 [0.5  1.   0.75]
18 10.999999997418142 [0.5  1.   0.75]
result of power_iteration: 11.000000000

(11.000000000704278, array([0.5 , 1.  , 0.75]), 19)

用随机矩阵测试，和 numpy 提供的特征值算法比较：

In [47]:
A = np.random.random((100, 100))
# print(A)

r = power_iteration(A, eps=1e-5)
print('by power_iteration:', r[0])

a = np.linalg.eig(A)[0]
ar = [x for x in a if not np.iscomplex(x)]
i = np.argmax(np.abs(ar))
print('by   np.linalg.eig:', ar[i])

by power_iteration: 50.445839029418046
by   np.linalg.eig: (50.44583922496655+0j)


- 反幂法

In [257]:
def inverse_iteration(A, m0=1, u0=None, eps=1e-8, max_steps=500, verbose=False):
    """反幂法（inverse iteration a.k.a. inverse power method）计算矩阵 A 的按模最小特征值、特征向量
    
    Args:
        A:   np_array_like 待求特征值的矩阵 (nxn)
        m0:  float 初始特征值
                default m0=1
        u0: np_array_like 初始特征向量（n）：要求无穷范式=1，通常取 u0 = (1, 1, ..., 1)
                default u0=None: 取 u0 = (1, 1, ..., 1)
        eps: float 精度要求
                default eps=1e-8
        max_steps: int 最大迭代次数
                default max_steps=1000
        verbose: bool, 若为 True 则打印出每一步的结果
                default verbose=False
        
    Returns:
        (m, u, k): 在 max_steps 次迭代以内得到第一组的满足 eps 的结果
            m: float 所求主特征值
            u: np.array 相应的特征向量
            k: int 迭代次数
        
    Raises:
        ValueError: 参数 A 不是方阵，
                    或 A 和给定 u0 尺寸不匹配
                    或 u0 = 0
        Exception:  无法在max_steps 次迭代以内得到满足精度 eps 的结果
    """
        
    _shape = np.shape(A)
    if len(_shape) < 2 or _shape[0] != _shape[1]:
        raise ValueError(f"unexpected A, shape: {_shape}")
    
    if u0 is not None:
        if len(u0) != _shape[0]:
            raise ValueError(f"A (shape={_shape}) and u0 (len={len(u0)}) not match")
        if np.all(u0 == 0):
            raise ValueError(f'bad u0: u0 == 0')
    else:  # not u0
        u0 = np.ones(_shape[0])
        
    m = m0
    u = u0

    for k in range(int(max_steps)):
        if verbose:
            print(k, 1/m, u)
        
        m_prev = m
        
        v = np.linalg.solve(A, u)
        mi = np.argmax(np.abs(v))
        m = v[mi]
        u = v / m
                
        if abs(m - m_prev) <= eps:
            break
        
    else:
        raise Exception(f"cannot reach eps ({eps}) after max_steps ({max_steps}). "
                        f"The last result: 1/m = {1/m}, u={u}")
    
    if verbose:
        print('result of inverse_iteration:', 1/m, u, k+1)
        
    return 1/m, u, k+1

In [258]:
A = [[2, 3, 2], [10, 3, 4], [3, 6, 1]]
inverse_iteration(A, eps=1e-5, max_steps=40, verbose=True)

0 1.0 [1. 1. 1.]
1 2.75 [-0.25        0.41666667  1.        ]
2 -1.7368421052631582 [-0.39473684 -0.25877193  1.        ]
3 -1.522306525037936 [-0.27587253 -0.28244815  1.        ]
4 -1.773137541073161 [-0.24854257 -0.33791831  1.        ]
5 -1.850713802809302 [-0.22932389 -0.36045702  1.        ]
6 -1.9067348353996336 [-0.21874548 -0.37508306  1.        ]
7 -1.9394212652843035 [-0.21209945 -0.38385382  1.        ]
8 -1.9604633754301686 [-0.20791023 -0.38945545  1.        ]
9 -1.973976092920489 [-0.20520426 -0.39306055  1.        ]
10 -1.9828015632828082 [-0.20343978 -0.3954137   1.        ]
11 -1.988599435522524 [-0.2022801  -0.39695986  1.        ]
12 -1.9924284508421255 [-0.20151431 -0.39798092  1.        ]
13 -1.9949649983827589 [-0.201007   -0.39865733  1.        ]
14 -1.996648958194817 [-0.20067021 -0.39910639  1.        ]
15 -1.997768464464223 [-0.20044631 -0.39940492  1.        ]
16 -1.998513415490028 [-0.20029732 -0.39960358  1.        ]
17 -1.999009434502497 [-0.20019811 -0.3

(-1.999942078533036, array([-0.20001158, -0.39998455,  1.        ]), 24)

基于实验 6 写的 LU 分解:

In [5]:
# lu 分解函数，来自 ex6/src/lu.py

def lu(a, sequence=False, swap_times: list = {}):
    """
    「高斯消去法」的 LU 分解.

    本函数可以计算「列主元高斯消元法」、「顺序高斯消元法」的 LU 分解，
    通过参数 sequence 控制，默认 sequence=False 使用「列主元高斯消元法」。

    Args:
        a: np_array_like 方阵 (nxn)
        sequence: bool, True 则使用顺序高斯消去法，False 为列主元的高斯消去法
            default: sequence=False
        swap_times: 这是一个**输出**用的变量，只有传入 dict 变量时才有效。
            若使用「列主元高斯消元法」（sequence=False）
            则，置 swap_times['swap_times'] = 行交换次数。
            这个值正常的输出中不需要，但在一些问题，比如，
            利用 LU 分解求行列式时，得到 swap_times 会很有帮助。

    Returns:
        (l, u, p): result

        l: np.array, Lower triangle result (nxn)
        u: np.array, Upper triangle result (nxn)
        p: np.array, Permutation: 交换后的行顺序 (n)
            p = None if sequence=True

    Raises:
        Exception: 存在为零的主元素
    """
    a = np.array(a, dtype=np.float)  # copy

    assert a.shape[0] == a.shape[1]
    n = a.shape[0]

    if not sequence:
        # p 记录行交换的过程，使用「列主元高斯消元法」才使用，否则为 None
        p = np.array([k for k in range(n)])
        # swap_times:  行交换次数
        if isinstance(swap_times, dict):
            swap_times['swap_times'] = 0
    else:
        p = None

    for k in range(n-1):
        if not sequence:
            i_max = k + np.argmax(np.abs(a[k:n, k]))

            if i_max != k:
                a[[i_max, k]] = a[[k, i_max]]  # swap rows
                p[[i_max, k]] = p[[k, i_max]]  # record
                swap_times['swap_times'] += 1

        if a[k][k] == 0:
            raise Exception("存在为零的主元素")

        for i in range(k+1, n):
            a[i][k] /= a[k][k]  # L @ 严格下三角
            for j in range(k+1, n):
                a[i][j] -= a[i][k] * a[k][j]  # U @ 上三角

    return np.tril(a, k=-1) + np.identity(a.shape[0]), np.triu(a), p


def solve_lu(b, l, u, p=None):
    """用 lu(a) 得到的 `pa=lu` 分解的结果求解原方程组 `ax=b` 的解 x。

    若 p 不为 None 则使用「列主元高斯消元」，p 为 None表示使用「顺序高斯消元」。

        # `@` means matrix multiplication, refer: https://docs.python.org/reference/expressions.html#binary-arithmetic-operations
        b = p @ b if p != None
        l @ y = b
        u @ x = y

    Args:
        b: np_array_like, 原方程组的右端常数（n）
        l: np_array_like, Lower triangle of lu_seq(a)
        u: np_array_like, Upper triangle of lu_seq(a)
        p: np_array_like, LU分解中交换后的行顺序
            default p=None: 未做行交换，即使用顺序高斯消去法

        使用列主元高斯消元法时，l, u, p 使用 lu(a) 得到的结果即可：
            solve_lu(b, *lu(a))
        或者使用顺序高斯消元：
            solve_lu(b, *lu(a, sequence=True))  # p=None

    Returns:
        x : np.array `ax=b` 的解（n）
    """
    assert np.shape(l) == np.shape(u)
    assert np.shape(l)[0] == np.shape(b)[0]

    n = np.shape(l)[0]

    # do swap
    if p is not None:
        b = [b[v] for v in p]

    # L * y = b
    y = np.zeros(n, dtype=np.float)
    y[0] = b[0]
    for i in range(1, n):
        bi = b[i]
        for j in range(0, i):
            bi -= y[j] * l[i][j]
        y[i] = bi / l[i][i]
    # print(y)

    # U * x = y
    x = np.zeros(n, dtype=np.float)
    x[n-1] = y[n-1] / u[n-1][n-1]
    for i in range(n-2, -1, -1):  # from n-2 (included) to 0 (included)
        yi = y[i]
        for j in range(i+1, n):
            yi -= x[j] * u[i][j]
        x[i] = yi / u[i][i]
    # print(x)

    return x

In [255]:
def inverse_iteration_lu(A, m0=1, u0=None, eps=1e-8, max_steps=500, verbose=False):
    """反幂法（inverse iteration a.k.a. inverse power method）计算矩阵特征值、特征向量
    
    基于 LU 分解
    
    Args:
        A:   np_array_like 待求特征值的矩阵 (nxn)
        m0:  float 初始特征值
                default m0=1
        u0: np_array_like 初始特征向量（n）：要求无穷范式=1，通常取 u0 = (1, 1, ..., 1)
                default u0=None: 取 u0 = (1, 1, ..., 1)
        eps: float 精度要求
                default eps=1e-8
        max_steps: int 最大迭代次数
                default max_steps=1000
        verbose: bool, 若为 True 则打印出每一步的结果
                default verbose=False
        
    Returns:
        (m, u, k): 在 max_steps 次迭代以内得到第一组的满足 eps 的结果
            m: float 所求主特征值
            u: np.array 相应的特征向量
            k: int 迭代次数
        
    Raises:
        ValueError: 参数 A 不是方阵，
                    或 A 和给定 u0 尺寸不匹配
                    或 u0 = 0
        Exception:  无法在max_steps 次迭代以内得到满足精度 eps 的结果
    """
        
    _shape = np.shape(A)
    if len(_shape) < 2 or _shape[0] != _shape[1]:
        raise ValueError(f"unexpected A, shape: {_shape}")
    
    if u0 is not None:
        if len(u0) != _shape[0]:
            raise ValueError(f"A (shape={_shape}) and u0 (len={len(u0)}) not match")
        if np.all(u0 == 0):
            raise ValueError(f'bad u0: u0 == 0')
    else:  # not u0
        u0 = np.ones(_shape[0])
        
    m = m0
    u = u0
    
    lupA = lu(A)

    for k in range(int(max_steps)):
        if verbose:
            print(k, 1/m, u)
        
        m_prev = m
        
        v = solve_lu(u, *lupA)
        mi = np.argmax(np.abs(v))
        m = v[mi]
        u = v / m
                
        if abs(m - m_prev) <= eps:
            break
        
    else:
        raise Exception(f"cannot reach eps ({eps}) after max_steps ({max_steps}). "
                        f"The last result: 1/m = {1/m}, u={u}")
    
    if verbose:
        print('result of inverse_iteration_lu:', 1/m, u, k+1)
        
    return 1/m, u, k+1

In [256]:
A = [[2, 3, 2], [10, 3, 4], [3, 6, 1]]
inverse_iteration_lu(A, eps=1e-5, max_steps=40, verbose=True)

0 1.0 [1. 1. 1.]
1 2.7499999999999996 [-0.25        0.41666667  1.        ]
2 -1.7368421052631577 [-0.39473684 -0.25877193  1.        ]
3 -1.5223065250379362 [-0.27587253 -0.28244815  1.        ]
4 -1.773137541073161 [-0.24854257 -0.33791831  1.        ]
5 -1.8507138028093015 [-0.22932389 -0.36045702  1.        ]
6 -1.906734835399633 [-0.21874548 -0.37508306  1.        ]
7 -1.939421265284303 [-0.21209945 -0.38385382  1.        ]
8 -1.9604633754301677 [-0.20791023 -0.38945545  1.        ]
9 -1.9739760929204886 [-0.20520426 -0.39306055  1.        ]
10 -1.9828015632828078 [-0.20343978 -0.3954137   1.        ]
11 -1.9885994355225236 [-0.2022801  -0.39695986  1.        ]
12 -1.992428450842125 [-0.20151431 -0.39798092  1.        ]
13 -1.994964998382758 [-0.201007   -0.39865733  1.        ]
14 -1.9966489581948166 [-0.20067021 -0.39910639  1.        ]
15 -1.997768464464223 [-0.20044631 -0.39940492  1.        ]
16 -1.998513415490028 [-0.20029732 -0.39960358  1.        ]
17 -1.9990094345024974 [

(-1.9999420785330355, array([-0.20001158, -0.39998455,  1.        ]), 24)

测试一下二者的运行速度：

In [79]:
# 测试运行速度

import time

def time_cost(func):
    """print time cost of a function func"""
    def wrapper(*args, **kwargs):
        s = time.time()
        ret = func(*args, **kwargs)
        e = time.time() - s
        print(f'{func.__name__} executed in {e: .4f} s')
        return ret
    return wrapper

eps = 1e-5
max_steps = 1e6

@time_cost
def inverse_iter_np_solve(A):
    return inverse_iteration(A, eps=eps, max_steps=max_steps)

@time_cost
def inverse_iter_lu_solve(A):
    return inverse_iteration_lu(A, eps=eps, max_steps=max_steps)

@time_cost
def call_numpy_linalg_eig(A):
    return np.linalg.eig(A)

In [85]:
# A = [[2, 3, 2], [10, 3, 4], [3, 6, 1]]

# 这里是随机的，不一定能算，有时候要多跑几次才能出来正确的结果
A = np.random.random((10, 10))
while np.linalg.det(A) == 0:
    print('na')
    A = np.random.random((10, 10))
# print(A)

r_eig = call_numpy_linalg_eig(A)
reals = [x for x in r_eig[0] if not np.iscomplex(x)]
i = np.argmin(np.abs(reals))
print(reals[i])

r_snp = inverse_iter_np_solve(A)
print(r_snp[0])

r_slu = inverse_iter_lu_solve(A)
print(r_slu[0])

assert str(r_snp[0])[:6] == str(r_slu[0])[:6], f'r_snp={r_snp[0]}, r_slu={r_slu[0]}'
assert str(r_slu[0])[:5] in str(r_eig[0]), f'r_slu ({r_slu[0]}) not in r_eig: {r_eig[0]}'

call_numpy_linalg_eig executed in  0.0003 s
(-0.034991233156967005+0j)
inverse_iter_np_solve executed in  0.0005 s
-0.034991232623709595
inverse_iter_lu_solve executed in  0.0031 s
-0.03499123262370956


In [374]:
import traceback

def origin_translation(A, p, x0=None, eps=1e-8, max_steps=1000, verbose=False):
    """原点位移算法
    (搜不到这个方法英文是什么)
    
    Args:
        A:  np_array_like, 待求特征值的矩阵 (nxn)
        p:  float 位移因子
        x0: np_array_like, 初始向量（n）, 要求 max(abs(x0)) == 1
                default x0=None: 取 x0 = (1, 1, ..., 1)
        eps: float, 控制精度
                default eps=1e-8
        max_steps: 最大迭代次数
                default max_steps=1000
        verbose: 打印出每步的结果，default verbose=False
    
    Returns:
        (eig, x, k)
            eig: float, A 的靠近 p 的特征值
            x: np.array, 单位化的特征向量
            k: int, 迭代次数
        
    Raises:
        ValueError: 参数 A 不是方阵，
                    或 A 和给定 x0 尺寸不匹配
                    或 x0 = 0
                    或 max(u0) != 1
        Exception:  无法在max_steps 次迭代以内得到满足精度 eps 的结果
    """
        
    _shape = np.shape(A)
    if len(_shape) < 2 or _shape[0] != _shape[1]:
        raise ValueError(f"unexpected A, shape: {_shape}")
    
    if x0 is not None:
        if len(x0) != _shape[0]:
            raise ValueError(f"A (shape={_shape}) and x0 (len={len(x0)}) not match")
        if np.all(x0 == 0):
            raise ValueError(f'bad x0: x0 == 0')
    else:  # not u0
        x0 = np.ones(_shape[0])
        
    _I = np.identity(_shape[0])
    
    eig = p
    x = x0
    
    for k in range(int(max_steps)):
        if verbose:
            print(k, eig, x)
        
        eig_prev = eig
        
        try:
            x = np.linalg.solve(A - eig * _I, x)
        except np.linalg.LinAlgError as e:
            # traceback.print_exc()
            x = np.linalg.solve(A - eig * _I + eps, x)
                
        q = np.argmax(np.abs(x))
        xq = x[q]
        
        eig = eig + 1 / xq

        
        x = x / xq
        
        if np.all(np.abs(eig - eig_prev) <= eps):
            break
    else:
        raise Exception(f"cannot reach eps ({eps}) after max_steps ({max_steps}). "
                                f"The last result: eig = {eig}, x={x}")
        
    if verbose:
        print('result of origin_translation:', eig, x, k+1)
        
    return eig, x, k+1

In [316]:
A = [[2, 3, 2], [10, 3, 4], [3, 6, 1]]
origin_translation(A, 10, verbose=True)

0 10 [1. 1. 1.]
1 10.709090909090909 [0.47272727 1.         0.74545455]
2 11.005485349227762 [0.50058964 1.         0.74989723]
3 11.000001815769398 [0.50000023 1.         0.74999988]
4 11.000000000000188 [0.5  1.   0.75]
result of origin_translation: 11.0 [0.5  1.   0.75] 5


(11.0, array([0.5 , 1.  , 0.75]), 5)

In [302]:
def accelerating_max_val_eig(A, x0=None, eps_pi=1e-1, eps_ot=1e-8, max_steps_pi=1000, max_steps_ot=1000, verbose=False):
    """动态原点位移算法求按模最大特征值
    
    Args:
        A:  np_array_like, 待求特征值的矩阵 (nxn)
        x0: np_array_like, 初始向量（n）, 要求 max(abs(x0)) == 1
                default x0=None: 取 x0 = (1, 1, ..., 1)
        eps_pi: float,「正幂法」迭代的控制精度
                default eps_pi=1e-1
        eps_ot: float,「原点位移」算法的控制精度，必须满足 eps > eps_ot
                default eps_ot=1e-8
        max_steps_pi: int,「正幂法」迭代的最大迭代次数
                default max_steps_pi=1000
        max_steps_ot: int,「原点位移」算法的最大迭代次数
                default max_steps_ot=1000
        verbose: bool:
                - False: 不打印任何过程信息;
                - True: 打印出「正幂法」、「原点位移」的完整迭代过程;
                default verbose=False
    
    Returns:
        (eig, eigv, k)
            eig: float, A 的按模最大特征值
            eigv: np.array, 单位化的特征向量
            k: 迭代次数
        
    Raises:
        ValueError: 参数 A 不是方阵，
                    或 A 和给定 x0 尺寸不匹配
                    或 x0 = 0
        Exception:  无法在max_steps 次迭代以内得到满足精度 eps 的结果
    """
    # 异常输入检测会在 power_iteration 中完成，此处不必重复辛劳
    
    if verbose:
        print("    power_iteration:")
    
    eig, eigv, k_pi = power_iteration(A, u0=x0, eps=eps_pi, max_steps=max_steps_pi, verbose=verbose)
    
    if verbose:
        print("    origin_translation:")
    
    eig, eigv, k_ot = origin_translation(A, eig, x0=eigv, eps=eps_ot, max_steps=max_steps_ot, verbose=verbose)
    
    return eig, eigv, k_pi + k_ot

In [303]:
A = [[2, 3, 2], [10, 3, 4], [3, 6, 1]]
accelerating_max_val_eig(A, verbose=True)

    power_iteration:
0 1 [1. 1. 1.]
1 17.0 [0.41176471 1.         0.58823529]
2 9.470588235294118 [0.52795031 1.         0.82608696]
3 11.58385093167702 [0.49276139 1.         0.72600536]
4 10.831635388739945 [0.50200485 1.         0.75743775]
5 11.049799514875502 [0.49945569 1.         0.74783731]
result of power_iteration: 10.985906091381928 [0.50014864 1.         0.75061668] 6
    origin_translation:
0 10.985906091381928 [0.50014864 1.         0.75061668]
1 10.999996033673042 [0.49999985 1.         0.74999938]
2 11.000000000001119 [0.5  1.   0.75]
result of origin_translation: 11.0 [0.5  1.   0.75] 3


(11.0, array([0.5 , 1.  , 0.75]), 9)

In [377]:
def accelerating_min_val_eig(A, x0=None, eps_ii=1e-1, eps_ot=1e-8, max_steps_ii=1000, max_steps_ot=1000, verbose=False):
    """动态原点位移算法求按模最小特征值
    
    Args:
        A:  np_array_like, 待求特征值的矩阵 (nxn)
        x0: np_array_like, 初始向量（n）, 要求 max(abs(x0)) == 1
                default x0=None: 取 x0 = (1, 1, ..., 1)
        eps_ii: float,「反幂法」迭代的控制精度
                default eps_ii=1e-1
        eps_ot: float,「原点位移」算法的控制精度，必须满足 eps > eps_ot
                default eps_ot=1e-8
        max_steps_ii: int,「反幂法」迭代的最大迭代次数
                default max_steps_ii=1000
        max_steps_ot: int,「原点位移」算法的最大迭代次数
                default max_steps_ot=1000
        verbose: bool:
                - False: 不打印任何过程信息;
                - True: 打印出「正幂法」、「原点位移」的完整迭代过程;
                default verbose=False
    
    Returns:
        (eig, eigv, k)
            eig: float, A 的按模最大特征值
            eigv: np.array, 单位化的特征向量
            k: 迭代次数
        
    Raises:
        ValueError: 参数 A 不是方阵，
                    或 A 和给定 x0 尺寸不匹配
                    或 x0 = 0
        Exception:  无法在max_steps 次迭代以内得到满足精度 eps 的结果
    """
    # 异常输入检测会在 power_iteration 中完成，此处不必重复辛劳
    
    if verbose:
        print("    inverse_iteration:")
    
    eig, eigv, k_ii = inverse_iteration(A, u0=x0, eps=eps_ii, max_steps=max_steps_ii, verbose=verbose)
    
    if verbose:
        print("    origin_translation:")
    
    eig, eigv, k_ot = origin_translation(A, eig, x0=eigv, eps=eps_ot, max_steps=max_steps_ot, verbose=verbose)
    
    return eig, eigv, k_ii + k_ot

In [378]:
A = [[2, 3, 2], [10, 3, 4], [3, 6, 1]]
accelerating_min_val_eig(A, verbose=True)

    inverse_iteration:
0 1.0 [1. 1. 1.]
1 2.75 [-0.25        0.41666667  1.        ]
2 -1.7368421052631582 [-0.39473684 -0.25877193  1.        ]
result of inverse_iteration: -1.522306525037936 [-0.27587253 -0.28244815  1.        ] 3
    origin_translation:
0 -1.522306525037936 [-0.27587253 -0.28244815  1.        ]
1 -1.8991931095340484 [-0.22075327 -0.37282221  1.        ]
2 -1.9913582399414025 [-0.20172412 -0.39769765  1.        ]
3 -1.9999267327390964 [-0.20001466 -0.39998046  1.        ]
4 -1.9999999946318703 [-0.2 -0.4  1. ]
result of origin_translation: -2.0 [-0.2 -0.4  1. ] 5


(-2.0, array([-0.2, -0.4,  1. ]), 8)

寻找一个较优的 eps_{pi, ii}:

In [306]:
A = [[2, 3, 2], [10, 3, 4], [3, 6, 1]]

print("max:")
for i in range(6):
    eps_i = 10 ** (-i)
    _, _, k = accelerating_max_val_eig(A, eps_pi=eps_i)
    print(i, k)
    
print("min:")
for i in range(6):
    eps_i = 10 ** (-i)
    _, _, k = accelerating_min_val_eig(A, eps_ii=eps_i)
    print(i, k)
    

max:
0 8
1 9
2 11
3 12
4 14
5 15
min:
0 7
1 8
2 11
3 15
4 21
5 26


对这个问题 eps_{pi, ii} 取 1 就比较好。

## 实验内容题目

In [376]:
A = [[4, -1, 1],
     [-1, 3, -2],
     [1, -2, 3]]

for u0 in [[1, 1, 1], [2.001, 1.999, 0], [0, 1, 1]]:
    print(f"\n-- 取初值 u0 = {u0}")

    print("\n正幂法：", end='')
    maxlp = power_iteration(A, u0=u0, eps=1e-6)
    print(maxlp)

    print("加速法：", end='')
    maxla = accelerating_max_val_eig(A, x0=u0, eps_pi=1e-3, eps_ot=1e-6)
    print(maxla)

    print("\n反幂法：", end='')
    minli = inverse_iteration(A, u0=u0, eps=1e-6)
    print(minli)

    print("加速法", end='')
    minla = accelerating_min_val_eig(A, x0=u0, eps_ii=1e-3, eps_ot=1e-6)
    print(minla)

print("\n-- 2.5 附近特征值：")
r = origin_translation(A, 2.5)
print("原点位移：", r)


-- 取初值 u0 = [1, 1, 1]

正幂法：(5.999999284744433, array([ 1.        , -0.99999982,  0.99999982]), 24)
加速法：(6.000000333333467, array([ 1.        , -0.99999947,  0.99999987]), 20)

反幂法：(1.0000004180226194, array([4.18175751e-07, 1.00000000e+00, 9.99999582e-01]), 13)
加速法(1.0000000000000009, array([-1.00994543e-15,  1.00000000e+00,  1.00000000e+00]), 9)

-- 取初值 u0 = [2.001, 1.999, 0]

正幂法：(5.999999476043831, array([ 1.        , -0.99999987,  0.99999987]), 35)
加速法：(5.999999666666533, array([ 1.        , -0.99999947,  0.99999987]), 30)

反幂法：(1.0000004184290925, array([4.18429144e-07, 1.00000000e+00, 9.99999582e-01]), 14)
加速法(1.0000000000000009, array([-9.81794114e-16,  1.00000000e+00,  1.00000000e+00]), 10)

-- 取初值 u0 = [0, 1, 1]

正幂法：(1, array([0., 1., 1.]), 1)
加速法：(0.9999999999999999, array([0., 1., 1.]), 4)

反幂法：(1.0, array([2.77555756e-17, 1.00000000e+00, 1.00000000e+00]), 1)
加速法(0.9999999999999999, array([0., 1., 1.]), 4)

-- 2.5 附近特征值：
原点位移： (3.0, array([ 1. ,  0.5, -0.5]), 5)
